In [49]:
import pandas as pd
import numpy as np
pd.set_option('future.no_silent_downcasting', True)

In [50]:
df_water = pd.read_excel("data/SuEatableLife_Food_Fooprint_database.xlsx", sheet_name = "SEL WF for users")
df_water = df_water.replace('-',np.nan)
df_co2 = pd.read_csv("data/Clean_CO2.csv")
df_prod = pd.read_csv("data/Production_Crops_Livestock_E_All_Data.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_10140\211997447.py:4: DtypeWarning: Columns (11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77,80,83,86,89,92,95,98,101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194,197) have mixed types. Specify dtype option on import or set low_memory=False.
  df_prod = pd.read_csv("data/Production_Crops_Livestock_E_All_Data.csv")


In [51]:
df_water.rename(columns={ 
                       "FOOD COMMODITY GROUP" : "food_commodity_group",
                       "Food commodity ITEM" : "food_commodity_item",  
             "Food commodity TYPOLOGY" : "food_commodity_typology",
             "Food commodity sub-TYPOLOGY": "Food_commodity_sub-TYPOLOGY",
                }, inplace= True)

In [52]:
print(len(df_co2["food_commodity_item"]))
print(len(df_water["food_commodity_item"]))

324
320


In [53]:
#lower all strings
df_water = df_water.map(lambda x: x.lower() if type(x) == str else x)
df_co2 = df_co2.map(lambda x: x.lower() if type(x) == str else x)
df_prod = df_prod.map(lambda x: x.lower() if type(x) == str else x)

In [54]:
def remove_asterix(df):
    '''
    We remove * as it corresponds to typologies having only one item in it and as it is different between the 2 datasets
    '''
    df.replace(to_replace = r"\*+",value=  '',inplace = True,regex=True)
    return df
def remove_s(df):
    '''
    Removing s at the end of words to avoid bananas not matching with banana
    '''
    df.replace({r'(\w+)(s)$' : r'\1'}, inplace = True,regex=True)
    return df


In [55]:
df_co2 = remove_asterix(df_co2)
df_water = remove_asterix(df_water)
df_water['food_commodity_item'] = remove_s(df_water['food_commodity_item'])
df_co2['food_commodity_item'] = remove_s(df_co2['food_commodity_item'])
df_prod['Item'] = remove_s(df_prod['Item'])

In [56]:
#remove lines where it is imported 
def remove_parenthesis_in_item(df, content_of_parenthesis):
    '''
    At the end of certain items there are parenthesis giving information on the item e.g (i) correspond to imported.
    However the others datasets don't have the same level of information and we just keep the main item if existing.
    '''
    df_imported = df[df['food_commodity_item'].str.contains(fr"\({content_of_parenthesis}\)")]
    for idx,row in df_imported.iterrows():
        item_to_find = row["food_commodity_item"][:-4]
        if df['food_commodity_item'].str.contains(item_to_find).sum() >1:
            df = df[df["food_commodity_item"] != row["food_commodity_item"]]
        else:
            # Replace using loc instead of replace to avoid warnings
            mask = df["food_commodity_item"] == row["food_commodity_item"]
            df.loc[mask, "food_commodity_item"] = item_to_find
    return df

# We remove those detailed informations but we might create small games to learn what consumes more between greenhouse or imported for example
df_co2 = remove_parenthesis_in_item(df_co2, "g")
df_co2 = remove_parenthesis_in_item(df_co2, "i")
df_water = remove_parenthesis_in_item(df_water, "f")
df_co2 = remove_parenthesis_in_item(df_co2, "G")
df_co2 = remove_parenthesis_in_item(df_co2, "f")
df_co2 = remove_parenthesis_in_item(df_co2, "l")
df_co2

C:\Users\User\AppData\Local\Temp\ipykernel_10140\2638439649.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if df['food_commodity_item'].str.contains(item_to_find).sum() >1:


,Unnamed: 0,food_commodity_group,food_commodity_item,CF_food_item,uncertainty,Suggested CF value,food_commodity_typology,CF_food_typology,Food_commodity_sub-TYPOLOGY,CF_food_sub_typology
0,0,agricultural processed,beer in can,0.693000,l,ok item,beer,0.69,NaN,NaN
1,1,agricultural processed,beer in glas,0.960500,l,ok item,beer,0.69,NaN,NaN
2,2,agricultural processed,beer modular can,0.530600,l,ok item,beer,0.69,NaN,NaN
3,3,agricultural processed,chocolate or cream filled cookie,1.533000,l,ok item,biscuits,1.47,NaN,NaN
4,4,agricultural processed,simple cookie,1.390750,l,ok item,biscuits,1.47,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
319,319,fishing,octopu,5.800000,h,better typol. or subtypol.,shellfish,5.67,cephalopod (sub-typ),5.80
320,320,fishing,squid,6.910000,h,better typol. or subtypol.,shellfish,5.67,cephalopod (sub-typ),5.80
321,321,fishing,prawns/shrimp,7.040000,l,ok item,shellfish,5.67,crustacean (sub-typ) (prawns/shrimps),7.04
322,322,fishing,octopus,7.878611,h,better typology,shellfish frozen,11.47,NaN,NaN


In [57]:
#TODO : rendre ça moins cheum
pd.options.mode.chained_assignment = None 

# First, let's identify the exact columns in each dataframe
co2_columns = df_co2.columns.tolist()
water_columns = df_water.columns.tolist()

# 1. Create indicator column during merge
co2_and_water_full = df_co2.merge(df_water, 
                            on=["food_commodity_item", "food_commodity_group"], 
                            how='outer',
                            indicator=True)

# 2. Get rows common to both dataframes
co2_and_water = co2_and_water_full[co2_and_water_full['_merge'] == 'both'].drop(columns=['_merge'])

# 3. Get rows unique to df_co2 - using only columns that actually exist in df_co2
df_only_co2 = df_co2[~df_co2.set_index(["food_commodity_item", "food_commodity_group"])
                      .index.isin(co2_and_water.set_index(["food_commodity_item", "food_commodity_group"])
                      .index)]

# 4. Get rows unique to df_water - using only columns that actually exist in df_water
df_only_water = df_water[~df_water.set_index(["food_commodity_item", "food_commodity_group"])
                        .index.isin(co2_and_water.set_index(["food_commodity_item", "food_commodity_group"])
                        .index)] 

df_asterisk_water = df_only_water[df_only_water["Suggested WF value"]=="item matching typology"]
df_asterisk_co2 =  df_only_co2[df_only_co2["Suggested CF value"]=="Item matching typology"]
df_different_co2 = df_only_co2.loc[df_only_co2.index.difference(df_asterisk_co2.index)]
df_different_water = df_only_water.loc[df_only_water.index.difference(df_asterisk_water.index)]

#Merge typologies with unique item in water with
for index,row in df_asterisk_water.iterrows():
    if row["food_commodity_typology"] in list(df_different_co2["food_commodity_typology"]):
        element = df_only_co2[df_only_co2["food_commodity_typology"]==row["food_commodity_typology"]]
        line_to_add = element
        
        line_to_add["CF_food_item"] = list(element["CF_food_typology"])[0]
        line_to_add["food_commodity_item"] = list(element["food_commodity_typology"])[0]
        #removing the detailed elements and keeping only the typology to get the same as for water 
        df_co2 = df_co2[df_co2["food_commodity_typology"]!=row["food_commodity_typology"]]
        df_co2 = pd.concat([df_co2, line_to_add], ignore_index=True)


In [58]:
def remove_item(df, item):
    return df[df["food_commodity_item"]!=item]
df_co2 = remove_item(df_co2, "cheese")
df_water = remove_item(df_water, "cheese")
#Remove stocks as it is just animals being moved and not production
df_prod = df_prod[df_prod["Element"]!="stocks"]
df_prod

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961,...,Y2020N,Y2021,Y2021F,Y2021N,Y2022,Y2022F,Y2022N,Y2023,Y2023F,Y2023N
0,2,'004,afghanistan,221,'01371,"almonds, in shell",5312,area harvested,ha,0.0,...,NaN,36862.0,a,NaN,36462.00,a,NaN,37000.00,a,NaN
1,2,'004,afghanistan,221,'01371,"almonds, in shell",5412,yield,kg/ha,NaN,...,NaN,1743.2,a,NaN,1742.00,a,NaN,1810.80,a,NaN
2,2,'004,afghanistan,221,'01371,"almonds, in shell",5510,production,t,0.0,...,NaN,64256.0,a,NaN,63515.00,a,NaN,67000.00,a,NaN
3,2,'004,afghanistan,711,'01654,"anise, badian, coriander, cumin, caraway, fenn...",5312,area harvested,ha,NaN,...,NaN,25357.0,e,NaN,25403.00,e,NaN,25439.00,e,NaN
4,2,'004,afghanistan,711,'01654,"anise, badian, coriander, cumin, caraway, fenn...",5412,yield,kg/ha,NaN,...,NaN,705.0,e,NaN,704.00,e,NaN,704.00,e,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78167,5817,'902,net food importing developing countries,1729,'f1729,"treenuts, total",5412,yield,kg/ha,614.5,...,NaN,683.0,e,NaN,676.60,e,NaN,682.70,e,NaN
78168,5817,'902,net food importing developing countries,1729,'f1729,"treenuts, total",5510,production,t,255724.0,...,NaN,2667688.9,e,NaN,2854981.94,e,NaN,2812938.79,e,NaN
78169,5817,'902,net food importing developing countries,1735,'f1735,vegetables primary,5312,area harvested,ha,1881425.0,...,NaN,7753692.0,e,NaN,7754677.00,e,NaN,7795058.00,e,NaN
78170,5817,'902,net food importing developing countries,1735,'f1735,vegetables primary,5412,yield,kg/ha,6617.1,...,NaN,12136.8,e,NaN,12442.00,e,NaN,12396.10,e,NaN


In [59]:
old_to_new_typo = pd.read_excel("data/old_to_new_typo.xlsx")
from_old_typo_to_new_typo = dict(zip(old_to_new_typo['old_typo'], old_to_new_typo['new_typo']))
df_co2["food_commodity_typology"] = df_co2["food_commodity_typology"].replace(from_old_typo_to_new_typo)
df_water["food_commodity_typology"] = df_water["food_commodity_typology"].replace(from_old_typo_to_new_typo)
df_water.loc[df_water["food_commodity_item"]=="palm nut","food_commodity_typology"] = "nuts"
df_co2

def avg_of_typology(df, col_to_avg):
    df[col_to_avg] = df.groupby("food_commodity_typology")[col_to_avg].transform('mean')
    df[col_to_avg] = round(df[col_to_avg], 2)
    return df
df_co2 = avg_of_typology(df_co2, "CF_food_typology")
df_water = avg_of_typology(df_water, "Water Footprint cc water/g o cc of food TYPOLOGY")
df_water

,food_commodity_group,food_commodity_item,Water Footprint liters water/kg o liter of food ITEM,Uncertainty low (L) high (H),Suggested WF value,food_commodity_typology,Water Footprint cc water/g o cc of food TYPOLOGY,Food_commodity_sub-TYPOLOGY,Water Footprint cc water/g o cc of food sub-TYPOLOGY
0,agricultural processed,apple juice,1140.0,l,ok item,drink,824.86,NaN,NaN
1,agricultural processed,barley flake,1977.0,l,ok item,breakfast cereals,2196.00,NaN,NaN
2,agricultural processed,beans,801.0,l,ok item,vegetables frozen,750.00,NaN,NaN
3,agricultural processed,beer,155.0,h,item matching typology,drink,824.86,NaN,NaN
4,agricultural processed,beet sugar,865.0,l,ok item,sugar,1294.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...
315,fishing,sturgeon,13420.0,l,ok item,fish,2313.00,NaN,NaN
316,fishing,tilapia,2263.0,l,ok item,fish,2313.00,NaN,NaN
317,fishing,trout,2188.0,l,ok item,fish,2313.00,NaN,NaN
318,fishing,turbot,1504.0,l,ok item,fish,2313.00,NaN,NaN


In [60]:
# Your original inner merge
co2_and_water = df_co2.merge(df_water, on = ["food_commodity_item","food_commodity_group","food_commodity_typology"], how='inner')
co2_and_water

,Unnamed: 0,food_commodity_group,food_commodity_item,CF_food_item,uncertainty,Suggested CF value,food_commodity_typology,CF_food_typology,Food_commodity_sub-TYPOLOGY_x,CF_food_sub_typology,Water Footprint liters water/kg o liter of food ITEM,Uncertainty low (L) high (H),Suggested WF value,Water Footprint cc water/g o cc of food TYPOLOGY,Food_commodity_sub-TYPOLOGY_y,Water Footprint cc water/g o cc of food sub-TYPOLOGY
0,3,agricultural processed,chocolate or cream filled cookie,1.53300,l,ok item,biscuits,1.47,NaN,NaN,2902.0,l,ok item,1870.00,NaN,NaN
1,4,agricultural processed,simple cookie,1.39075,l,ok item,biscuits,1.47,NaN,NaN,1723.0,l,ok item,1870.00,NaN,NaN
2,6,agricultural processed,bread multicereal,0.70300,h,better typology,bread,0.98,NaN,NaN,771.0,l,better typology,1006.20,NaN,NaN
3,7,agricultural processed,bread plain,0.88500,l,ok item,bread,0.98,NaN,NaN,1031.0,l,ok item,1006.20,NaN,NaN
4,8,agricultural processed,bread whole,0.77200,l,ok item,bread,0.98,NaN,NaN,887.0,l,ok item,1006.20,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,0,agricultural processed,beer,0.69000,l,ok item,drink,1.47,NaN,NaN,155.0,h,item matching typology,824.86,NaN,NaN
133,1,agricultural processed,beer,0.69000,l,ok item,drink,1.47,NaN,NaN,155.0,h,item matching typology,824.86,NaN,NaN
134,2,agricultural processed,beer,0.69000,l,ok item,drink,1.47,NaN,NaN,155.0,h,item matching typology,824.86,NaN,NaN
135,95,agricultural processed,wine,0.66000,l,ok item,drink,1.47,NaN,NaN,524.0,l,item matching typology,824.86,NaN,NaN


## Transform to hierarchical json


In [61]:

import json

def convert_df_to_hierarchical_json(df,value):
    # Create the root node with 'children' for each unique food_commodity_group
    result = {"name": "Food", "children": []}
    
    # Group by the first level (food_commodity_group)
    for group_name, group_df in df.groupby('food_commodity_group'):
        group_node = {"name": group_name, "children": []}
        
        # Group by the second level (food_commodity_typology)
        for typology_name, typology_df in group_df.groupby('food_commodity_typology'):
            typology_node = {"name": typology_name, "children": []}
            
            # Group by the third level (food_commodity_item)
            for item_name, item_df in typology_df.groupby('food_commodity_item'):
                # If there are multiple CF_food_item values for a single item, 
                # we take the average or any other aggregation you prefer
                cf_value = item_df[value].mean()
                
                item_node = {"name": item_name, "value": round(float(cf_value),2)}
                typology_node["children"].append(item_node)
            
            group_node["children"].append(typology_node)
        
        result["children"].append(group_node)
    
    return json.dumps(result, indent=2)

co2_json = convert_df_to_hierarchical_json(co2_and_water,'CF_food_item')
# Writing to sample.json
with open("data/data_carbon.json", "w") as outfile:
    outfile.write(co2_json)
water_json = convert_df_to_hierarchical_json(co2_and_water,'Water Footprint liters water/kg o liter of food ITEM')
# Writing to sample.json
with open("data/data_water.json", "w") as outfile:
    outfile.write(water_json)


There are 135 items in common.
Problem to resolve : different orders of words and subtlities between products.

## Merging with df_prod (unfinished)

In [62]:
# Your original inner merge
all_df_merged = co2_and_water.merge(df_prod, left_on = "food_commodity_item",right_on= "Item",  how='inner')
all_df_merged

,Unnamed: 0,food_commodity_group,food_commodity_item,CF_food_item,uncertainty,Suggested CF value,food_commodity_typology,CF_food_typology,Food_commodity_sub-TYPOLOGY_x,CF_food_sub_typology,...,Y2020N,Y2021,Y2021F,Y2021N,Y2022,Y2022F,Y2022N,Y2023,Y2023F,Y2023N
0,64,agricultural processed,olive oil,3.265,l,ok item,oil,2.11,NaN,NaN,...,NaN,1043.00,i,NaN,1050.00,i,NaN,NaN,NaN,NaN
1,64,agricultural processed,olive oil,3.265,l,ok item,oil,2.11,NaN,NaN,...,unofficial figure,16500.00,x,unofficial figure,15500.00,x,unofficial figure,NaN,NaN,NaN
2,64,agricultural processed,olive oil,3.265,l,ok item,oil,2.11,NaN,NaN,...,unofficial figure,71000.00,x,unofficial figure,88200.00,x,unofficial figure,NaN,NaN,NaN
3,64,agricultural processed,olive oil,3.265,l,ok item,oil,2.11,NaN,NaN,...,unofficial figure,30000.00,x,unofficial figure,33000.00,a,NaN,NaN,NaN,NaN
4,64,agricultural processed,olive oil,3.265,l,ok item,oil,2.11,NaN,NaN,...,unofficial figure,22600.00,x,unofficial figure,20000.00,x,unofficial figure,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8437,96,agricultural processed,wine,0.660,h,better typology,drink,1.47,NaN,NaN,...,NaN,7787.29,e,NaN,7279.47,e,NaN,NaN,NaN,NaN
8438,96,agricultural processed,wine,0.660,h,better typology,drink,1.47,NaN,NaN,...,NaN,378076.85,e,NaN,378175.21,e,NaN,NaN,NaN,NaN
8439,96,agricultural processed,wine,0.660,h,better typology,drink,1.47,NaN,NaN,...,NaN,16228.00,e,NaN,14214.00,e,NaN,NaN,NaN,NaN
8440,96,agricultural processed,wine,0.660,h,better typology,drink,1.47,NaN,NaN,...,NaN,27202.37,e,NaN,26719.03,e,NaN,NaN,NaN,NaN


In [63]:
len(all_df_merged["food_commodity_item"].unique())

25

In [64]:
all_df_merged["food_commodity_item"].unique()

array(['olive oil', 'palm oil', 'date', 'orange', 'apple', 'apricot',
       'avocado', 'banana', 'fig', 'grape', 'pear', 'pineapple',
       'watermelon', 'olive', 'currant', 'barley', 'millet', 'oat', 'rye',
       'sorghum', 'wheat', 'sunflower seed', 'spinach', 'artichoke',
       'wine'], dtype=object)

13 elements in common between the 3 datasets T-T
26 in commons when removing s.

In [71]:
def df_without_common(df ,column_name):
    return df[~df[column_name].isin(list(all_df_merged["food_commodity_item"]))]

def export_to_txt_the_column(df ,column_name :str,export_name :str):
    df = df_without_common(df ,column_name).drop_duplicates(subset = column_name)
    df[column_name].to_csv(f'data/{export_name}.txt', sep=" ", index=False)


export_to_txt_the_column(df_prod ,"Item","df_prod")
#export_to_txt_the_column(df_co2 ,"food_commodity_item","df_co2")
#export_to_txt_the_column(df_water ,"food_commodity_item","df_water")

In [66]:
export_to_txt_the_column(df_prod ,"Item","df_prod")

In [70]:
df_to_export_co2 =df_without_common(df_co2 ,"food_commodity_item")
df_to_export_co2 = df_to_export_co2.sort_values(by= "food_commodity_item",axis=0, ascending=True)

In [ ]:
df_without_common(df_co2 ,"food_commodity_item").to_excel("df_co2.xlsx")
df_without_common(df_water ,"food_commodity_item").to_excel("df_water.xlsx")

PermissionError: [Errno 13] Permission denied: 'df_co2.xlsx'

In [ ]:
common = pd.read_csv("food-commodity-mapping-csv(1).txt")

In [ ]:
common = common[~common["prod.txt"].str.contains("-")]

In [ ]:
common.to_excel("data/commmon.xlsx")

In [ ]:
from_co2_to_common = dict(zip(common['co2_item.txt'], common['Shortest Name']))
df_co2["food_commodity_item"] = df_co2["food_commodity_item"].replace(from_co2_to_common)
df_co2

,Unnamed: 0,food_commodity_group,food_commodity_item,CF_food_item,uncertainty,Suggested CF value,food_commodity_typology,CF_food_typology,Food_commodity_sub-TYPOLOGY,CF_food_sub_typology
0,0,AGRICULTURAL PROCESSED,BEER IN CAN,0.693000,L,OK item,BEER,0.69,NaN,NaN
1,1,AGRICULTURAL PROCESSED,BEER IN GLASS,0.960500,L,OK item,BEER,0.69,NaN,NaN
2,2,AGRICULTURAL PROCESSED,BEER MODULAR CAN,0.530600,L,OK item,BEER,0.69,NaN,NaN
3,3,AGRICULTURAL PROCESSED,CHOCOLATE OR CREAM FILLED COOKIES,1.533000,L,OK item,BISCUITS,1.47,NaN,NaN
4,4,AGRICULTURAL PROCESSED,SIMPLE COOKIES,1.390750,L,OK item,BISCUITS,1.47,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
319,319,FISHING,OCTOPUS,5.800000,H,better typol. or subtypol.,SHELLFISH,5.67,CEPHALOPOD (SUB-TYP),5.80
320,320,FISHING,SQUID,6.910000,H,better typol. or subtypol.,SHELLFISH,5.67,CEPHALOPOD (SUB-TYP),5.80
321,321,FISHING,PRAWNS/SHRIMPS,7.040000,L,OK item,SHELLFISH,5.67,CRUSTACEAN (SUB-TYP) (Prawns/Shrimps),7.04
322,322,FISHING,OCTOPUS (F),7.878611,H,better typology,SHELLFISH FROZEN,11.47,NaN,NaN


In [ ]:
from_prod_to_common = dict(zip(common['prod.txt'], common['Shortest Name']))
df_prod["food_commodity_item"] = df_prod["food_commodity_item"].replace(from_co2_to_common)

KeyError: 'food_commodity_item'

In [ ]:
#Question quand dans un des df c'est 2 sous catégorie ou prend la moyenne puis on développe après?